In [ ]:
print("The following is for exercise 5 part ii")

The following is for exercise 5 part ii


In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
import numpy
numpy.random.seed(1)
a = numpy.random.randn(128)

In [ ]:
# Convert values from double precision to single precision (requirement for CUDA)
a = a.astype(numpy.float32)
sub_array = a[0::4]
sub_array = list(sub_array)
sub_array = numpy.array(sub_array)

In [ ]:
# Allocate memory in device
a_gpu = cuda.mem_alloc(sub_array.nbytes)

In [ ]:
# Copy array data from a to a_gpu (from host to device)
cuda.memcpy_htod(a_gpu, sub_array)

In [ ]:
# Create CUDA kernel
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = blockIdx.x*blockDim.x + threadIdx.x;
    
    a[idx] *= 2;
  }
  """)

In [ ]:
# Here we choose the block and grid dimensions
# TODO: add grid dimensions
grid_dim = (1,1)
# TODO: add block dimensions
block_dim = (32,1,1)
func = mod.get_function("doublify")
func(a_gpu, block=block_dim, grid = grid_dim)

In [ ]:
sub_array_doubled = numpy.empty_like(sub_array)
cuda.memcpy_dtoh(sub_array_doubled, a_gpu)


In [ ]:
# We now have the doubled 32 elements 

# We need to now put them back into the original array, a

# But for testing purposes, lets keep the orginal array a and make a copy of it

a_copy = a

a_copy[0::4] = sub_array_doubled

# we'll print to see if every 4th element has been doubled 
print("Original values")
print(a[:10])

print("4th doubled values")
print(a_copy[:10])